# Предварительная обработка данных

In [22]:
import os
import sys
import pandas
import numpy as np

In [23]:
def save_clean_data(cleaner, X_train, y_train, X_test, folder='default'):
    clean_folder_path = './clean/'
    path = clean_folder_path + folder
    if not os.path.exists(path):
        os.makedirs(path)

    y_train.to_csv(path + '/y_train.csv')
    cleaner(X_train).to_csv(path + '/X_train.csv')
    cleaner(X_test).to_csv(path + '/X_test.csv')

In [24]:
def clean(X):
    """ Меняем пропуски на 0 """
    return X.fillna(0)

In [25]:
def clean_category(X):
    """ Удаляем 11 категориальных признаков """
    X = clean(X)
    del X['lobby_type']
    for n in range(1, 6):
        del X['r{}_hero'.format(n)]
        del X['d{}_hero'.format(n)]

    return X

In [26]:
def hero_bag(X):
    """ Формируем мешок слов по героям """
    heroes = pandas.read_csv('./dictionaries/heroes.csv')
    X_pick = np.zeros((X.shape[0], len(heroes)))
    for i, match_id in enumerate(X.index):
        for p in range(5):
            X_pick[i, X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

    return pandas.DataFrame(X_pick, index=X.index)

### Загружаем данные

In [27]:
train = pandas.read_csv('./features.csv', index_col='match_id')
test = pandas.read_csv('./features_test.csv', index_col='match_id')

### Удаляем признаки, связанные с итогами матча

In [28]:
train.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire'
        ], axis=1, inplace=True)

### Разделяем датасет на признаки и целевую переменную

##### Как называется столбец, содержащий целевую переменную?

radiant_win - итог матча: 1, если победила команда Radiant, 0 — иначе

In [29]:
X = train
y = train['radiant_win'].to_frame()
del train['radiant_win']

In [30]:
save_clean_data(clean, X, y, test, folder="clean")

In [31]:
save_clean_data(clean_category, X, y, test, folder='clean_category')

In [32]:
save_clean_data(hero_bag, X, y, test, folder='hero_bag')